# Cracking the Code: How OpenAI’s O1-Mini Outshines GPT-4o in Trade Processing - Ashish Kumar Jain

## O Series Magic 
### The idea here is to generate an application when we have a high-level design and want to have a starting point to get ready for a production-ready app. We will generate the app using both GPT4 and O1 models and see what magic O1 reasoning capability is adding to the app.

## The model used in Code Generation

### O Series Model (O1- mini) - I will use the o1-mini reasoning model as I still cannot access the o3-mini in my account. As per Open AI Access to o3-mini is rolling out to select API users across trust tiers 3 through 5. They hope to rapidly expand access in the days and weeks ahead. 

### GPT Model (GPT-4o-mini ) - GPT-4o-mini is a lighter version, optimised for efficiency and cost-effectiveness, likely meant for faster responses with lower computational power. It will help us save some money while running this exercise.


#### We will use python code for illustration purpose. You can use this code for your applications. You also need to install below required python library to run the code.

In [ ]:
#pip install --upgrade openai

#### We must load the OpenAI key from the environment variable and set it into api_key. We will get this key while registering with OPENAI. We then instantiate the Open AI client using the key. We will also define two variables for two models.

In [ ]:
import openai
import os
from openai import OpenAI
openai.api_key  = os.getenv('OPENAI_API_KEY')
client = OpenAI(api_key=openai.api_key)
GPT_4O_MINI_MODEL = 'gpt-4o-mini'
O1_MINI_MODEL = 'o1-mini'

#### Let's create some helper function to show the formatted code output in the Jupyter Notebook.

In [ ]:
import re
from IPython.display import display, Markdown

def extract_code_blocks(text):
    pattern = r'```(\w+)?\n(.*?)```'
    matches = re.findall(pattern, text, re.DOTALL)
    
    code_blocks = {}
    for i, (lang, code) in enumerate(matches):
        lang = lang.strip() if lang else "plaintext"
        title = f"Code Block {i+1} ({lang})"
        code_blocks[title] = f"```{lang}\n{code.strip()}\n```"
    
    return code_blocks

def format_code(raw_text):
    code_blocks = extract_code_blocks(raw_text)
    for title, code in code_blocks.items():
        display(Markdown(f"### {title}\n"))
        display(Markdown(code))

## Trade Ordering App

### In the financial industry, trading apps allow users to buy, sell, and manage financial instruments such as stocks, bonds, commodities, forex, cryptocurrencies, etc. At the heart of this app is its micro-service architecture style, which has a collection of loosely coupled, independently deployable services, each of which is responsible for a specific business function. These services normally talk to each other using some kind of Enterprise Service Bus (ESB), which sends events to this BUS. This kind of architecture  provides scalability, reliability, and speed in modern trading apps.

### We will have three microservices written in different languages. We will tie them together into event-driven architecture using Kafka. We will ask to write stubs for Kafka producers and consumers where we can add our business logic later. We also only generate the Microservices and ESB using Kafka, and for simplicity, we will not have UI elements and other components.

### Let's draft our prompt for the model.

In [ ]:
trade_prompt = """
Design a trade order management system (TOMS) with the following microservices:

order-service (Node.js & Express): Accepts trade orders via REST API, validates them, and publishes to Kafka.

risk-service (Python & FastAPI): Listens to new orders, applies pre-trade risk checks, and updates an in-memory risk book.

execution-service (Node.js & Express): Picks up risk-approved trades, simulates execution, and sends execution reports.

Use PostgreSQL for trade persistence and Redis for real-time risk book updates.
Include meaningful stubs for Kafka producers/consumers, but don't implement actual business logic.

Respond with the code only! Nothing else!"""

### Let's run the first with the GPT4 model using OPENAI chat completion API and format code output and print that.

In [ ]:
trade_gpt4_response = client.chat.completions.create(model=GPT_4O_MINI_MODEL,
                                                     messages=[{"role":"user","content": trade_prompt}])
trade_gpt4_code = trade_gpt4_response.choices[0].message.content
format_code(trade_gpt4_code)

### Now, let's run the same trade prompt on the o1-mini model and see the response.

In [ ]:
trade_o1_response = client.chat.completions.create(model=O1_MINI_MODEL,
                                                     messages=[{"role":"user","content": trade_prompt}])
trade_o1_code = trade_o1_response.choices[0].message.content
format_code(trade_o1_code)

### We learned earlier that one of the use cases for o1 is Evaluation and benchmarking for other model responses. Now, we can use the o1-mini model to pass the output of both models and do a comparison analysis of both models.

#### Now compare the output of both models using o1-mini.

In [ ]:
better_trade_code_promot = f"which code is better and why? Option 1: {trade_gpt4_code}... or Option 2: {trade_o1_code}"
better_trade_response = client.chat.completions.create(model=O1_MINI_MODEL,
                                                     messages=[{"role":"user","content": better_trade_code_promot}])
better_trade = better_trade_response.choices[0].message.content

### Let's print the output in the markdown format.

In [ ]:
from IPython.display import display, Markdown
display(Markdown(better_trade))